In [1]:
# Llama3 for Inference
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
torch.random.manual_seed(0)

In [2]:
from transformers import BitsAndBytesConfig
# Create a BitsAndBytesConfig object with your desired settings
quant_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [3]:

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B", 
    device_map="auto", 
    torch_dtype="auto", 
    #trust_remote_code=True, 
    quantization_config=quant_config
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
last_cmd_examples = [
    {
        "user": "last",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -a",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -i",
        "example": "john   pts/1        192.168.1.10\njane   pts/2        192.168.1.11"
    },
    {
        "user": "last -F",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30:45 2024   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45:12 2024   gone - no logout"
    },
    {
        "user": "last -R",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -x",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in"
    }
]


nc_cmd_examples = [
    {
        "user": "nc",
        "example": "Connected to hospital-server.local on port 80"
    },
    {
        "user": "nc -l",
        "example": "Listening on port 8080 for incoming connections"
    },
    {
        "user": "nc -v hospital-server.local 443",
        "example": "Connection to hospital-server.local 443 succeeded"
    },
    {
        "user": "nc -u -l 1234",
        "example": "Listening on UDP port 1234"
    },
    {
        "user": "nc -zv hospital-db.local 3306",
        "example": "Connection to hospital-db.local 3306 succeeded"
    }
]

netstat_cmd_examples = [
    {
        "command": "netstat",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:https       ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "command": "netstat -a",
        "example": "Active Internet connections (servers and established)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 *:ssh                   *:*                     LISTEN\ntcp        0      0 localhost:smtp          *:*                     LISTEN\nudp        0      0 *:123                   *:*"
    },
    {
        "command": "netstat -l",
        "example": "Active Internet connections (only servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 *:ssh                   *:*                     LISTEN\nudp        0      0 *:123                   *:*"
    },
    {
        "command": "netstat -p",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:https       ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "command": "netstat -n",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:443         ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "command": "netstat -r",
        "example": "Kernel IP routing table\nDestination     Gateway         Genmask         Flags   MSS Window  irtt Iface\ndefault         router          0.0.0.0         UG        0 0          0 eth0\n192.168.1.0     0.0.0.0         255.255.255.0   U         0 0          0 eth0"
    },
    {
        "command": "netstat -s",
        "example": "Ip:\n    12345 total packets received\n    0 forwarded\n    0 incoming packets discarded\n    0 incoming packets discarded\n    0 incoming packets discarded\n    0 fragments dropped after final reassembly\n    0 fragments dropped after final reassembly"
    }
]


nohup_cmd_examples = [
    {
        "command": "nohup command &",
        "example": "nohup python patient_monitor.py &"
    },
    {
        "command": "nohup command > output.log &",
        "example": "nohup bash schedule_generator.sh > schedule.log &"
    },
    {
        "command": "nohup command > output.log 2>&1 &",
        "example": "nohup ./patient_report.sh > patient_report.log 2>&1 &"
    },
    {
        "command": "nohup command -p PID &",
        "example": "nohup java -jar hospital_mgmt.jar -p 1234 &"
    }
]


scp_cmd_examples =[
    {
        "command": "scp",
        "example": "patients.txt user@example.com:/home/remote_user/data"
    },
    {
        "command": "scp -r",
        "example": "records/ user@example.com:/backup"
    },
    {
        "command": "scp -P",
        "example": "patients.txt -P 2222 user@example.com:/home/remote_user/data"
    },
    {
        "command": "scp -v",
        "example": "patients.txt user@example.com:/home/remote_user/data\nDebugging connection..."
    },
    {
        "command": "scp -i",
        "example": "patients.txt -i ~/.ssh/hospital_key user@example.com:/home/remote_user/data"
    },
    {
        "command": "scp -B",
        "example": "patients.txt user@example.com:/backup\nBackground transfer initiated."
    },
    {
        "command": "scp -C",
        "example": "patients.txt user@example.com:/home/remote_user/data\nCompression enabled."
    },
    {
        "command": "scp -l",
        "example": "patients.txt -l 5000 user@example.com:/home/remote_user/data\nLimited bandwidth transfer."
    }]


service_cmd_examples = [
    {
        "command": "service",
        "example": "apache2 start"
    },
    {
        "command": "service apache2 status",
        "example": "● apache2.service - The Apache HTTP Server\n   Loaded: loaded (/lib/systemd/system/apache2.service; enabled; vendor preset: enabled)\n   Active: active (running) since Mon 2024-07-08 10:00:00 UTC; 1h ago\n     Docs: https://httpd.apache.org/docs/2.4/\n Main PID: 12345 (apache2)\n    Tasks: 55 (limit: 512)\n   Memory: 150.0M\n   CGroup: /system.slice/apache2.service\n           ├─12345 /usr/sbin/apache2 -k start\n           ├─12346 /usr/sbin/apache2 -k start\n           └─12347 /usr/sbin/apache2 -k start"
    },
    {
        "command": "service mysql start",
        "example": "Starting MySQL database server: mysqld."
    },
    {
        "command": "service nginx restart",
        "example": "Restarting nginx (via systemctl): nginx.service."
    },
    {
        "command": "service --status-all",
        "example": " [ + ]  apache2\n [ - ]  mysql\n [ ? ]  nginx"
    }
]

tftp_cmd_examples = [
    {
        "command": "tftp",
        "example": "Usage: tftp [OPTION]... HOST [PORT]\n"
    },
    {
        "command": "tftp -c get patients.txt",
        "example": "Received 2048 bytes in 0.1 seconds"
    },
    {
        "command": "tftp -c put doctors.txt",
        "example": "Sent 4096 bytes in 0.2 seconds"
    },
    {
        "command": "tftp -m binary -c get schedule.csv",
        "example": "Received 1024 bytes in 0.1 seconds"
    }
]

ulimit_cmd_examples = [
    {
        "command": "ulimit",
        "example": "unlimited"
    },
    {
        "command": "ulimit -a",
        "example": "core file size          (blocks, -c) 0\ndata seg size           (kbytes, -d) unlimited\nscheduling priority             (-e) 0\nfile size               (blocks, -f) unlimited\npending signals                 (-i) 31529\nmax locked memory       (kbytes, -l) 64\nmax memory size         (kbytes, -m) unlimited\nopen files                      (-n) 1024\npipe size            (512 bytes, -p) 8\nPOSIX message queues     (bytes, -q) 819200\nreal-time priority              (-r) 0\nstack size              (kbytes, -s) 8192\ncpu time               (seconds, -t) unlimited\nmax user processes              (-u) 31529\nvirtual memory          (kbytes, -v) unlimited\nfile locks                      (-x) unlimited"
    },
    {
        "command": "ulimit -c",
        "example": "0"
    },
    {
        "command": "ulimit -u",
        "example": "31529"
    },
    {
        "command": "ulimit -n",
        "example": "1024"
    }
]


unzip_cmd_examples = [
    {
        "command": "unzip",
        "example": "patient_records.zip"
    },
    {
        "command": "unzip -l",
        "example": "    2048  2024-07-08 09:00   patients/patient1.txt\n    4096  2024-07-08 09:00   patients/patient2.txt\n    1024  2024-07-08 09:00   schedule/schedule.csv"
    },
    {
        "command": "unzip -t",
        "example": "No errors detected in compressed data of patient_records.zip."
    },
    {
        "command": "unzip -d",
        "example": "Extracting to directory 'patient_records/'"
    },
    {
        "command": "unzip -o",
        "example": "Overwrite existing files without prompting."
    },
    {
        "command": "unzip -q",
        "example": ""
    }
]


wget_cmd_examples = [
    {
        "command": "wget https://example.com/patients.csv",
        "example": "--2024-07-08 10:00:00--  https://example.com/patients.csv\n" +
                   "Resolving example.com (example.com)... 192.0.2.1\n" +
                   "Connecting to example.com (example.com)|192.0.2.1|:443... connected.\n" +
                   "HTTP request sent, awaiting response... 200 OK\n" +
                   "Length: 2048 (2.0K) [text/csv]\n" +
                   "Saving to: ‘patients.csv’\n\n" +
                   "100%[======================================>] 2,048       --.-K/s   in 0.001s\n\n" +
                   "2024-07-08 10:00:01 (2.00 MB/s) - ‘patients.csv’ saved [2048/2048]"
    },
    {
        "command": "wget -r -np -nH --cut-dirs=1 ftp://example.com/records/",
        "example": "--2024-07-08 10:05:00--  ftp://example.com/records/\n" +
                   "           => ‘example.com/records/index.html’\n" +
                   "Resolving example.com (example.com)... 192.0.2.1\n" +
                   "Connecting to example.com (example.com)|192.0.2.1|:21... connected.\n" +
                   "Logging in as anonymous ... Logged in!\n" +
                   "==> SYST ... done.    ==> PWD ... done.\n" +
                   "==> TYPE I ... done.  ==> CWD /records ... done.\n" +
                   "==> PASV ... done.    ==> LIST ... done.\n" +
                   "Mode ...                \n" +
                   "Length ...              \n" +
                   "Saving to: ‘example.com/records/index.html’\n" +
                   "\n" +
                   "     [ <=>                                   ] 2,048       --.-K/s   in 0.001s\n\n" +
                   "2024-07-08 10:05:01 (2.00 MB/s) - ‘example.com/records/index.html’ saved [2048]"
    },
    {
        "command": "wget -qO- https://example.com/doctors.txt",
        "example": "Dr. John Doe\nDr. Jane Smith\nDr. Michael Johnson\n"
    }
]

yum_cms_examples = [
    {
        "command": "yum",
        "example": "Installed Packages:\nhealthcare-app-1.2.3-1.el7.x86_64\nmedication-db-2024.1-1.el7.noarch\nAvailable Packages:\nhealthcare-app-1.2.4-1.el7.x86_64\nemergency-kit-1.0-1.el7.noarch"
    },
    {
        "command": "yum list",
        "example": "Installed Packages:\nhealthcare-app-1.2.3-1.el7.x86_64\nmedication-db-2024.1-1.el7.noarch\nAvailable Packages:\nhealthcare-app-1.2.4-1.el7.x86_64\nemergency-kit-1.0-1.el7.noarch"
    },
    {
        "command": "yum install healthcare-app",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\nResolving Dependencies\n--> Running transaction check\n---> Package healthcare-app.x86_64 0:1.2.4-1.el7 will be installed\n--> Finished Dependency Resolution\n\nDependencies Resolved\n\n====================================================================================================\n Package           Arch             Version                Repository                           Size\n====================================================================================================\nInstalling:\n healthcare-app    x86_64           1.2.4-1.el7            hospital-repo                       10 M\n\nTransaction Summary\n====================================================================================================\nInstall  1 Package\n\nTotal download size: 10 M\nInstalled size: 20 M\nIs this ok [y/d/N]: y\nDownloading packages:\nRunning transaction check\nRunning transaction test\nTransaction test succeeded\nRunning transaction\n  Installing : healthcare-app-1.2.4-1.el7.x86_64                                                  1/1 \n  Verifying  : healthcare-app-1.2.4-1.el7.x86_64                                                  1/1 \n\nInstalled:\n  healthcare-app.x86_64 0:1.2.4-1.el7                                                               \n\nComplete!"
    },
    {
        "command": "yum update",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\nNo packages marked for update"
    },
    {
        "command": "yum remove medication-db",
        "example": "Loaded plugins: fastestmirror\nResolving Dependencies\n--> Running transaction check\n---> Package medication-db.noarch 0:2024.1-1.el7 will be erased\n--> Finished Dependency Resolution\n\nDependencies Resolved\n\n====================================================================================================\n Package             Arch             Version                  Repository                         Size\n====================================================================================================\nRemoving:\n medication-db       noarch           2024.1-1.el7             @hospital-repo                    5.0 k\n\nTransaction Summary\n====================================================================================================\nRemove  1 Package\n\nInstalled size: 5.0 k\nIs this ok [y/N]: y\nDownloading packages:\nRunning transaction check\nRunning transaction test\nTransaction test succeeded\nRunning transaction\n  Erasing    : medication-db-2024.1-1.el7.noarch                                                1/1 \n  Verifying  : medication-db-2024.1-1.el7.noarch                                                1/1 \n\nRemoved:\n  medication-db.noarch 0:2024.1-1.el7                                                           \n\nComplete!"
    },
    {
        "command": "yum search emergency",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\n=============================== N/S matched: emergency ===============================\nemergency-kit.noarch : Emergency kit for hospital facilities\n\n  Name and summary matches only, use 'search all' for everything.\n"
    }
]



In [5]:

command_examples_list =  [
    nc_cmd_examples,
    netstat_cmd_examples,
    nohup_cmd_examples,
    scp_cmd_examples,
    service_cmd_examples,
    tftp_cmd_examples,
    ulimit_cmd_examples,
    unzip_cmd_examples,
    wget_cmd_examples,
    yum_cms_examples,
    last_cmd_examples
                         ]

command_example =[unzip_cmd_examples]

In [7]:

system_str = "You are Linux OS terminal for a server containing sensitive patient data." \
          "Your personality is: You are a Linux OS terminal. You act and respond exactly as a Linux terminal." \
          "You will respond to all commands just as a Linux terminal would." \
          "You can only respond to user inputs and you must not write any commands on your own." \
          "You must not in any case have a conversation with user as a chatbot and must not explain your output and do not repeat commands user inputs." \
          "Do not explain to user what they are seeing. Only respond as Linux terminal." \
          "You will need to make up realistic answers to the command, as they would be returned by a real linux terminal for a hospital server." \
          "It is very important that you do not name files and directiories file1.txt file2.txt file3.txt or similarly, rather create plausible file names for a real terminal with patient data."


#system_str = "You are a Linux Server in a hospital. Output a response for the given command input."


In [10]:

for command_examples in command_examples_list:
    for command_example in command_examples:

        content_str = command_example['command'] + " Use the following as an example for the output. " +  command_example['example']

        messages = [
            {"role": "system", "content": system_str},
            {"role": "user", "content": command_example['command'] }
        ]
        
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            chat_template=chat_template
        )
        
        generation_args = {
            "max_new_tokens": 600,
            "return_full_text": False,
            "temperature": 0.3,
            "do_sample": False,
        }
        
        output = pipe(messages, **generation_args)
        
        print("---------------------------------------------------------")
        print("User")
        print(command_example['command'])
        print("Assistant")
        print(output[0]['generated_text'])
        

ValueError: This model has multiple chat templates with no default specified! Please either pass a chat template or the name of the template you wish to use to the `chat_template` argument. Available template names are ['messages'].